In [ ]:
!pip install langchain langchain-community langchain-openai faiss-cpu sentence-transformers pdfplumber PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4

In [ ]:
import nltk
nltk.download() # d -> all

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
import os
import fitz  # PyMuPDF is still used for creating the dummy PDF
from langchain_community.document_loaders import PyPDFLoader, PDFPlumberLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.docstore.document import Document # To create new Document objects
from nltk.tokenize import sent_tokenize, word_tokenize
# from langchain_openai import ChatOpenAI

In [ ]:
dummy_pdf_path = "AI_powered_integration_of_multi_source_data_for_TAA_discovery_to.pdf"
# doc = fitz.open()
# page1 = doc.new_page()
# page1.insert_text((72, 72), "The Solar System: A Brief Overview")
# page1.insert_text((72, 108), "The solar system consists of the Sun and everything that orbits it. This includes eight planets and their moons, dwarf planets, asteroids, and comets.")
# page1.insert_text((72, 144), "Mars, known as the Red Planet, is the fourth planet from the Sun. It has a thin atmosphere and two small moons, Phobos and Deimos.")

# page2 = doc.new_page()
# page2.insert_text((72, 72), "Jupiter and its Moons")
# page2.insert_text((72, 108), "Jupiter is the largest planet in our solar system. It is a gas giant, primarily composed of hydrogen and helium.")
# page2.insert_text((72, 144), "It has a famous Great Red Spot, which is a giant storm raging for centuries. Jupiter has dozens of moons, with the four largest being the Galilean moons: Io, Europa, Ganymede, and Callisto.")

# doc.save(dummy_pdf_path)
# doc.close()
# print(f"INFO: Created a dummy PDF for demonstration: '{dummy_pdf_path}'")

In [ ]:
def load_pdf_documents(file_path):
  document_loader = PDFPlumberLoader(file_path)
  return document_loader.load()

In [ ]:
raw_docs = load_pdf_documents(dummy_pdf_path)

In [ ]:
raw_docs[0].metadata

{'source': 'AI_powered_integration_of_multi_source_data_for_TAA_discovery_to.pdf',
 'file_path': 'AI_powered_integration_of_multi_source_data_for_TAA_discovery_to.pdf',
 'page': 0,
 'total_pages': 9,
 'Author': 'Xie, Tao',
 'Comments': '',
 'Company': '',
 'CreationDate': "D:20250506175120-07'00'",
 'Creator': 'Acrobat PDFMaker 11 for Word',
 'Keywords': '',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_ActionId': '296b9d1c-3a3b-4187-9aa1-0048fda51a4c',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_ContentBits': '0',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_Enabled': 'true',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_Method': 'Privileged',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_Name': '4791b42f-c435-42ca-9531-75a3f42aae3d',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_SetDate': '2025-03-20T21:56:47Z',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3d_SiteId': '7a916015-20ae-4ad1-9170-eefd915e9272',
 'MSIP_Label_4791b42f-c435-42ca-9531-75a3f42aae3

In [ ]:
raw_docs[0].page_content

'bioRxiv preprint doi: https://doi.org/10.1101/2025.05.06.652559; this version posted May 8, 2025. The copyright holder for this preprint (which\nwas not certified by peer review) is the author/funder, who has granted bioRxiv a license to display the preprint in perpetuity. It is made\navailable under aCC-BY-NC-ND 4.0 International license.\nAI-powered integration of multi-source data for TAA discovery to\naccelerate ADC and TCE drug development (I):\nTAA Target Identification and Prioritization\nTao Xie* & Chao-Hui Huang*\n*: co-first authors\nEmail: Tao.Xie@pfizer.com\nChao-Hui.Huang@pfizer.com\nAbstract:\nThe advancement of T-cell engagers (TCEs) and antibody-drug conjugates (ADCs) has been\nhindered by fragmented data landscapes. This paper, the first in a series, introduces an AI-driven\nframework specifically for tumor-associated antigen (TAA) target identification and prioritization,\na critical initial step in TCE and ADC development. Our framework integrates diverse datasets—\

In [ ]:
print("INFO: Splitting document into sentences...")
sentence_docs = []
for doc in raw_docs:
    # Extract content and metadata from the original document (page)
    content = doc.page_content
    metadata = doc.metadata.copy() # Make a copy to avoid modifying the original



    # sentences = content.split(".")

    sentences = re.split(r'(?<=[.!?])\s+', content.strip())

    # Filter out empty strings
    sentences = [s for s in sentences if s.strip() and len(s) > 3 and " " in s]

    # Create a new Document for each sentence with the same metadata
    for sentence in sentences:
        # new_sent = sentence.replace("\n", " ")
        # if new_sent.strip() == "":
        #     continue
        new_doc = Document(page_content=sentence.replace("\n", " "), metadata=metadata)
        sentence_docs.append(new_doc)

print(f"INFO: Created {len(sentence_docs)} sentence-level documents.")

INFO: Splitting document into sentences...
INFO: Created 141 sentence-level documents.


In [ ]:
for sent in sentence_docs:
  print(sent.page_content)


bioRxiv preprint doi: https://doi.org/10.1101/2025.05.06.652559; this version posted May 8, 2025.
The copyright holder for this preprint (which was not certified by peer review) is the author/funder, who has granted bioRxiv a license to display the preprint in perpetuity.
It is made available under aCC-BY-NC-ND 4.0 International license.
AI-powered integration of multi-source data for TAA discovery to accelerate ADC and TCE drug development (I): TAA Target Identification and Prioritization Tao Xie* & Chao-Hui Huang* *: co-first authors Email: Tao.Xie@pfizer.com Chao-Hui.Huang@pfizer.com Abstract: The advancement of T-cell engagers (TCEs) and antibody-drug conjugates (ADCs) has been hindered by fragmented data landscapes.
This paper, the first in a series, introduces an AI-driven framework specifically for tumor-associated antigen (TAA) target identification and prioritization, a critical initial step in TCE and ADC development.
Our framework integrates diverse datasets— including multi

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
import re
from PyPDF2 import PdfReader

# Load the PDF
pdf_path = "AI_powered_integration_of_multi_source_data_for_TAA_discovery_to.pdf"
reader = PdfReader(pdf_path)

# Extract text from all pages
full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

# Split into sentences (basic approach using regex)
# This will split on ., ?, or ! followed by a space or end of text
sentences = re.split(r'(?<=[.!?])\s+', full_text.strip())

# Filter out empty strings
sentences = [s for s in sentences if s.strip()]

len(sentences)


147